Christian Sutton   
9/19/20  

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt
SRdata = pd.read_csv("C:/Users/physi/Desktop/AppliedMachineLearning_EN.705.601/Mod3/master.csv")

In [2]:
SRdata.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


# Problem 1
**[20 pts] Due to the severity of this real-world crisis, what information would be the most important one to   "machinelearn"? Can it be learned?**  

After reveiwing this dataset, I believe the most important information to machine learn are the features "age", "generation", "sex" and "suicides/100k pop" There seems to be correlation between suicides/100k pop and the other mentioned  features when viewing a correlation matrix. These correlations in the data between the features should help to produce a working model. Yes, when powerful features are selected from this dataset, a machine learning model can certainly be built and tested.

# Problem 2  
**[20 pts] Explain in detail how one should set up the problem? Would it be a regression or a classification problem? Any unsupervised approach to see some patterns exist or not?**  

The first step in building a model is simply to look at the available data to see what features are available to be learned. In this first step I see there are twelve features and 27820 data rows.  Second, step is to check for any missing values, erronous data points, and what data types the structure has. I see that one feature, "HDI for year" is missing nearly 2/3 of its values. In my judgement, engineering 2/3 of the values for "HDI for year" does not seem useful when there are other features to machine learn on. In cases like this, I would opt to remove that entire feature and not use it in the model. I this case I will be removing 'HDI for year" from the model, simply due to the amount of missing values. Additionally, after graphng each country with "Year" X "suicides/100k pop" X "age". I see that there are a list of countries who are simply reporting zero suicides for most years. I don't want to contiminate my dataset with data from countries who are not reporting accurate information. This is another judgement call on my part but it is intuitive to me that zero suicides are highly unlikly in a country's population. I will process the data and remove these countries that seem to be offending the dataset in this manner. An option would be to input statisitically meaningful values where I felt a country was constantly reporting zeros but in that case I again have the view that I don't want to contaminate the model on purpose with statistical noise unless I have to. For the third step, I would preprocess any categorial features into numerical ones. I have to take care to make sure to use ordinal features or nominal ones where appropriate. Forth, I would then build a correlation matrix to assess the correlations between the features. I want to build a model based on powerful features to reduce the amount of noise the model in contending with. As stated above, It seems the features having the best correlation with "suicides/100k pop" are "age", "generation"and "sex". I am modeling this as a classification problem. I have taken "suicides/100k pop" and binned it into two groups to build a dependent variable. One group is 17.73 per 100K and less and the other group is greater than 17.73 per 100K, where 17.73 represents the the top quartile of suicides per 100k. 


In [3]:
SRdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
country               27820 non-null object
year                  27820 non-null int64
sex                   27820 non-null object
age                   27820 non-null object
suicides_no           27820 non-null int64
population            27820 non-null int64
suicides/100k pop     27820 non-null float64
country-year          27820 non-null object
HDI for year          8364 non-null float64
 gdp_for_year ($)     27820 non-null object
gdp_per_capita ($)    27820 non-null int64
generation            27820 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


In [4]:
#Plot each country "year" X "suicides/100k pop" X age

def plot_bc(_df):  # Make sure use a '_variable' name to avoid shadowing variables in other cells
    g = sns.FacetGrid(_df, col='country', hue='age')
    g.map(plt.scatter, 'year', 'suicides/100k pop', alpha=.7)
    g.add_legend()
    plt.show()

#plot_bc(SRdata)

In [5]:
# Looking at the data above about how countries are reporting data, my judgement is to remove these countries entire 
# datasets from this model. I don't want this model corrupted by these countries reporting numerous zeroes for all ages

# Antigua and Barbuda
# Aruba
# Bosnia and Herzegovina
# Cabo Verde
# Cyprus
# Dominica
# Grenada
# Jamaica
# Kuwait
# Maldives
# Montenegro
# Oman
# Qatar
# Saint Kitts and Nevis

In [6]:
# removing these countries from the data
SRdata = SRdata[SRdata.country != "Antigua and Barbuda"]
SRdata = SRdata[SRdata.country != "Aruba"]
SRdata = SRdata[SRdata.country != "Bosnia and Herzegovina"]
SRdata = SRdata[SRdata.country != "Cabo Verdea"]
SRdata = SRdata[SRdata.country != "Cyprus"]
SRdata = SRdata[SRdata.country != "Dominica"]
SRdata = SRdata[SRdata.country != "Grenada"]
SRdata = SRdata[SRdata.country != "Jamaica"]
SRdata = SRdata[SRdata.country != "Kuwait"]
SRdata = SRdata[SRdata.country != "Maldives"]
SRdata = SRdata[SRdata.country != "Montenegro"]
SRdata = SRdata[SRdata.country != "Oman"]
SRdata = SRdata[SRdata.country != "Qatar"]
SRdata = SRdata[SRdata.country != "Saint Kitts and Nevis"]

In [7]:
# Since HDI for year is missing most of it's values, this feature will be removed
#Country-year is redundant to features "year" and "country"
SRdata =SRdata.drop(["HDI for year","country-year"], axis =1)

In [8]:
# These are the unique values for the string features
for col in SRdata.columns:
    if SRdata[col].dtype == np.object:
        print(col, SRdata[col].unique())

country ['Albania' 'Argentina' 'Armenia' 'Australia' 'Austria' 'Azerbaijan'
 'Bahamas' 'Bahrain' 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Brazil'
 'Bulgaria' 'Cabo Verde' 'Canada' 'Chile' 'Colombia' 'Costa Rica'
 'Croatia' 'Cuba' 'Czech Republic' 'Denmark' 'Ecuador' 'El Salvador'
 'Estonia' 'Fiji' 'Finland' 'France' 'Georgia' 'Germany' 'Greece'
 'Guatemala' 'Guyana' 'Hungary' 'Iceland' 'Ireland' 'Israel' 'Italy'
 'Japan' 'Kazakhstan' 'Kiribati' 'Kyrgyzstan' 'Latvia' 'Lithuania'
 'Luxembourg' 'Macau' 'Malta' 'Mauritius' 'Mexico' 'Mongolia'
 'Netherlands' 'New Zealand' 'Nicaragua' 'Norway' 'Panama' 'Paraguay'
 'Philippines' 'Poland' 'Portugal' 'Puerto Rico' 'Republic of Korea'
 'Romania' 'Russian Federation' 'Saint Lucia'
 'Saint Vincent and Grenadines' 'San Marino' 'Serbia' 'Seychelles'
 'Singapore' 'Slovakia' 'Slovenia' 'South Africa' 'Spain' 'Sri Lanka'
 'Suriname' 'Sweden' 'Switzerland' 'Thailand' 'Trinidad and Tobago'
 'Turkey' 'Turkmenistan' 'Ukraine' 'United Arab Emirates' 'United

# Problem 3
**[20 pts] What should be the dependent variable?**  

My dependent variable is a new feature created by binning the "suicides/100k pop" feature. The value of bin one is [0-17.73] and the value of bin two is (17.73-inf). As also stated above, I'm looking for a classifer that find high suicide rate versus low suicide rate. The new variable will be called "chance of suicide?" with 1 being equal to true and 0 equal to false

# Problem 4  
**[20 pts] Rank the variables to find some strong correlations between the independent variables and the dependent variable you decided.**  


I looks like The features I find with the most correlation are sex, age, generation, chance of suicide. The rest of the features never strongly correlate with the dependent variable. I'll build the model with these high correlation features. 


In [9]:
# # change gdp_for_year ($) to int data type for correlation 

# #later turned this off due to run time on this calulation. This feature isn't using anyway

# SRdata.astype({" gdp_for_year ($) ": "str"})
# for i in range(0,25810):
#     SRdata.iloc[i,7] = int(SRdata.iloc[i,7].replace(",",""))
# SRdata.astype({" gdp_for_year ($) ": "int64"})
# ;

In [10]:
#removing this feature because it isn't used later and processing it causes long run times
SRdata =SRdata.drop([" gdp_for_year ($) "], axis =1)

In [11]:
#binning my dependent variable

cut_labels = ['Low chance', 'High Chance']
cut_bins = [-.1, 17.73, 2000]
SRdata['Chance of suicide'] = pd.cut(SRdata['suicides/100k pop'], bins=cut_bins, labels=cut_labels)

In [12]:
#OneHotEncode all category features (for exploration of data)
SRdata1 =pd.get_dummies(SRdata)

In [13]:
# correlation of OneHotEncoded data (for exploration of data)
SRdata1.corr()

,year,suicides_no,population,suicides/100k pop,gdp_per_capita ($),country_Albania,country_Argentina,country_Armenia,country_Australia,country_Austria,...,age_55-74 years,age_75+ years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials,generation_Silent,Chance of suicide_Low chance,Chance of suicide_High Chance
year,1.000000,-0.001064,0.015262,-0.035937,3.288064e-01,-0.023022,-1.634744e-02,2.103485e-02,-1.841941e-02,-1.050456e-02,...,7.902650e-04,7.902650e-04,-6.722889e-02,-3.571466e-01,-1.058393e-01,2.684748e-01,2.132608e-01,7.499919e-02,0.035404,-0.035404
suicides_no,-0.001064,1.000000,0.613366,0.303101,6.622833e-02,-0.027636,-5.221702e-03,-2.950517e-02,-8.480661e-03,-1.709626e-02,...,5.926709e-02,-5.256732e-02,1.155535e-01,-2.310793e-02,-2.859334e-03,-6.267716e-02,-8.020154e-02,2.457519e-02,-0.267908,0.267908
population,0.015262,0.613366,1.000000,-0.010856,8.872236e-02,-0.044061,2.409391e-02,-4.621383e-02,-1.404116e-02,-4.088522e-02,...,6.518801e-03,-1.515661e-01,1.029240e-01,-9.644469e-02,3.606610e-02,-7.593517e-03,-1.906581e-03,-5.558568e-02,0.006617,-0.006617
suicides/100k pop,-0.035937,0.303101,-0.010856,1.000000,6.032501e-03,-0.053405,-1.973591e-02,-5.805276e-02,-3.808861e-03,6.467560e-02,...,8.226603e-02,2.756173e-01,4.864916e-02,2.000657e-01,-7.031946e-02,-1.575876e-01,-2.100035e-01,1.699749e-01,-0.755645,0.755645
gdp_per_capita ($),0.328806,0.066228,0.088722,0.006033,1.000000e+00,-0.080437,-5.670881e-02,-8.543147e-02,1.016275e-01,1.144231e-01,...,1.748261e-04,1.748261e-04,-2.140425e-02,-1.095877e-01,-3.586482e-02,1.070370e-01,5.819767e-02,2.107470e-02,-0.061530,0.061530
country_Albania,-0.023022,-0.027636,-0.044061,-0.053405,-8.043726e-02,1.000000,-1.229336e-02,-1.098693e-02,-1.209060e-02,-1.245994e-02,...,-4.577896e-05,-4.577896e-05,4.996284e-04,4.467938e-03,2.661524e-03,-9.989414e-03,-4.861822e-03,3.674569e-03,0.057788,-0.057788
country_Argentina,-0.016347,-0.005222,0.024094,-0.019736,-5.670881e-02,-0.012293,1.000000e+00,-1.306971e-02,-1.438260e-02,-1.482196e-02,...,-5.445722e-05,-5.445722e-05,9.020772e-04,7.183044e-03,1.484225e-03,-2.046882e-03,-4.547498e-03,-1.975544e-03,0.020996,-0.020996
country_Armenia,0.021035,-0.029505,-0.046214,-0.058053,-8.543147e-02,-0.010987,-1.306971e-02,1.000000e+00,-1.285415e-02,-1.324681e-02,...,2.755873e-04,2.755873e-04,2.247822e-03,-7.151761e-03,-2.538885e-03,4.011581e-03,5.146066e-03,-1.495737e-03,0.059870,-0.059870
country_Australia,-0.018419,-0.008481,-0.014041,-0.003809,1.016275e-01,-0.012091,-1.438260e-02,-1.285415e-02,1.000000e+00,-1.457750e-02,...,-5.355904e-05,-5.355904e-05,1.053662e-03,8.624678e-03,2.116791e-03,-1.150308e-03,-5.835860e-03,-3.011040e-03,-0.030549,0.030549
country_Austria,-0.010505,-0.017096,-0.040885,0.064676,1.144231e-01,-0.012460,-1.482196e-02,-1.324681e-02,-1.457750e-02,1.000000e+00,...,2.316730e-04,2.316730e-04,1.058006e-03,6.017739e-03,1.226182e-03,-2.772922e-03,-2.979464e-03,-2.158735e-03,-0.058944,0.058944


In [14]:
## remove low correlation columns
SRdata =SRdata.drop(["country","year","gdp_per_capita ($)","population","suicides/100k pop","suicides_no"], axis =1)

In [15]:
# encode ordinal values to age, suicide chance, 
ages = {'15-24 years':20, '5-14 years':10,'25-34 years':30,'35-54 years':45,'55-74 years':65,'75+ years':75}
Generations = {'G.I. Generation':1,'Silent':2,'Boomers':3,\
               'Generation X':4,'Millenials':5,'Generation Z':6}
Chance= {'High Chance':1,'Low chance':0}
SRdata['age']= SRdata['age'].map(ages)
SRdata['generation']= SRdata['generation'].map(Generations)
SRdata['Chance of suicide']= SRdata['Chance of suicide'].map(Chance)

In [16]:
### OneHotEncode all category features after editing columns. This is the data set to be used

SRdata =pd.get_dummies(SRdata)

In [17]:
SRdata.corr()

,age,generation,Chance of suicide,sex_female,sex_male
age,1.000000,-9.137420e-01,0.308577,0.000000e+00,0.000000e+00
generation,-0.913742,1.000000e+00,-0.295489,-8.226813e-19,8.226813e-19
Chance of suicide,0.308577,-2.954890e-01,1.000000,-4.383149e-01,4.383149e-01
sex_female,0.000000,-8.226813e-19,-0.438315,1.000000e+00,-1.000000e+00
sex_male,0.000000,8.226813e-19,0.438315,-1.000000e+00,1.000000e+00


In [18]:
SRdata.head()

,age,generation,Chance of suicide,sex_female,sex_male
0,20,4,0,0,1
1,45,2,0,0,1
2,20,4,0,1,0
3,75,1,0,0,1
4,30,3,0,0,1


# Problem 5  
**[20 pts] Pre-process the dataset and list the major features you want to use. Note that not all features are crucial. For example, country-year variable is a derived feature and for a classifier it would not be necessary to include the year, the country and the country-year together. In fact, one must avoid adding a derived feature and the original at the same time.
List the independent features you want to use.**   

The data has been preprocessed above....

**The features that i'm going to use are:**  
age                    
Chance of suicide -----Dependent variable             
sex_female                    
sex_male                      
generation            

# Problem 6
**[20 pts bonus] Devise a classification problem and present a prototype model.
Note that we will continue to this problem in the following modules.**  

The model that will be used is a random forest as shown below. Note: other models are commented out 

In [19]:
#split data, scale data, check sizes

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X, Y =SRdata.iloc[:,[0,1,3,4]], SRdata.iloc[:,2]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=.2,random_state=0)

SS = StandardScaler()
X_TrainScaled = SS.fit_transform(X_train)
X_testScaled = SS.transform(X_test)
X_TrainScaled.shape, X_testScaled.shape, y_train.shape, y_test.shape

((20648, 4), (5162, 4), (20648,), (5162,))

In [20]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#accuracy .80, f1score .53
# LinSv = LinearSVC(random_state=0, tol=1e-5)
# LinSv.fit(X_TrainScaled,y_train)
# pred = LinSv.predict(X_testScaled)

#This model will be used
#accuracy .81, f1score .67
RF = RandomForestClassifier(max_depth=5, random_state=0)
RF.fit(X_TrainScaled,y_train)
pred = RF.predict(X_testScaled)

#accuracy .81, f1score .62
# sv = SVC(gamma='auto')
# sv.fit(X_TrainScaled,y_train)
# pred = sv.predict( X_testScaled)

C:\Users\physi\Anaconda4\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [21]:
pred.shape, y_test.shape

((5162,), (5162,))

In [22]:
y_test =np.array(y_test)

In [23]:
# had some issue with dimensions so I reconfigured back into a dataframe for processing
y_test =np.array(y_test)
p = {'predicted':pred}
actual= pd.DataFrame(data= y_test)
results = pd.DataFrame(data=p,)
results['actual']= actual
results.head()

,predicted,actual
0,0,0
1,0,0
2,0,0
3,1,1
4,1,1


In [24]:
from sklearn.metrics import f1_score,accuracy_score

In [25]:
f1_score(results.iloc[:,1],results.iloc[:,0])

0.6721536351165981

In [26]:
accuracy_score(results.iloc[:,1],results.iloc[:,0])

0.8148004649360713